In [1]:
from pyhive import hive
from ucimlrepo import fetch_ucirepo
import pandas as pd

In [2]:
conn = hive.Connection(
    host='localhost',
    port=10000,
    username='maria_dev'
)

cursor = conn.cursor()

def hive_query(query):
    cursor.execute(query)
    for result in cursor.fetchall():
        print(result)

hive_query('SHOW TABLES')

('credit_approval',)
('credit_fraud',)
('cuba',)
('movie_names',)
('ratings',)


In [15]:
adult = fetch_ucirepo(id=2)
X = adult.data.features
y = adult.data.targets

df = pd.concat([X, y], axis=1)
df.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [16]:
cursor.execute("""
               CREATE DATABASE IF NOT EXISTS adult
               """)

In [17]:
# Save DataFrame to CSV
df.to_csv('adult_data.csv', index=False, header=False)

In [ ]:
# Create external table pointing to the CSV
cursor.execute("""
CREATE EXTERNAL TABLE IF NOT EXISTS adult.adult_data (
    age INT,
    workclass STRING,
    fnlwgt INT,
    education STRING,
    education_num INT,
    marital_status STRING,
    occupation STRING,
    relationship STRING,
    race STRING,
    sex STRING,
    capital_gain INT,
    capital_loss INT,
    hours_per_week INT,
    native_country STRING,
    income STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/user/maria_dev/adult/'
""")

# Then load the data (run this in Hive shell or via subprocess)
# LOAD DATA LOCAL INPATH 'adult_data.csv' INTO TABLE adult.adult_data
# make sure that there is only 1 file inside adult folder

In [24]:
cursor.execute("""
LOAD DATA INPATH 'adult_data.csv' INTO TABLE adult.adult_data
               """)

OperationalError: TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=["*org.apache.hive.service.cli.HiveSQLException:Error while compiling statement: FAILED: SemanticException Line 2:17 Invalid path ''adult_data.csv'': No files matching path hdfs://sandbox-hdp.hortonworks.com:8020/user/hive/adult_data.csv:17:16", 'org.apache.hive.service.cli.operation.Operation:toSQLException:Operation.java:324', 'org.apache.hive.service.cli.operation.SQLOperation:prepare:SQLOperation.java:148', 'org.apache.hive.service.cli.operation.SQLOperation:runInternal:SQLOperation.java:228', 'org.apache.hive.service.cli.operation.Operation:run:Operation.java:264', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatementInternal:HiveSessionImpl.java:479', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatement:HiveSessionImpl.java:454', 'org.apache.hive.service.cli.CLIService:executeStatement:CLIService.java:285', 'org.apache.hive.service.cli.thrift.ThriftCLIService:Execute

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529 entries, 0 to 528
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   adult_data.age             529 non-null    int64 
 1   adult_data.workclass       529 non-null    object
 2   adult_data.fnlwgt          529 non-null    int64 
 3   adult_data.education       529 non-null    object
 4   adult_data.education_num   529 non-null    int64 
 5   adult_data.marital_status  529 non-null    object
 6   adult_data.occupation      529 non-null    object
 7   adult_data.relationship    529 non-null    object
 8   adult_data.race            529 non-null    object
 9   adult_data.sex             529 non-null    object
 10  adult_data.capital_gain    529 non-null    int64 
 11  adult_data.capital_loss    529 non-null    int64 
 12  adult_data.hours_per_week  529 non-null    int64 
 13  adult_data.native_country  529 non-null    object
 14  adult_data

In [19]:
df.describe()

,adult_data.age,adult_data.fnlwgt,adult_data.education_num,adult_data.capital_gain,adult_data.capital_loss,adult_data.hours_per_week
count,529.000000,5.290000e+02,529.000000,529.000000,529.000000,529.000000
mean,38.018904,1.956842e+05,10.079395,576.627599,104.591682,39.631380
std,13.480191,1.147115e+05,2.566046,2669.324322,432.105263,11.895785
min,17.000000,2.117400e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.157450e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.840180e+05,10.000000,0.000000,0.000000,40.000000
75%,46.000000,2.496090e+05,12.000000,0.000000,0.000000,42.000000
max,90.000000,1.033222e+06,16.000000,34095.000000,2415.000000,98.000000


In [20]:
df.isnull().sum().sum()

0

In [21]:
df['adult_data.workclass'].value_counts()

adult_data.workclass
Private             365
Local-gov            39
Self-emp-not-inc     36
?                    35
State-gov            22
Federal-gov          17
Self-emp-inc         15
Name: count, dtype: int64

In [10]:
import pandas_datareader.wb as wb
import pandas as pd

# Define indicators
indicators = {
    'NY.GDP.PCAP.CD': 'gdp_per_capita',
    'SE.ADT.LITR.ZS': 'literacy_rate',
    'SE.PRM.ENRR': 'primary_enrollment',
    'SE.SEC.ENRR': 'secondary_enrollment',
    'SE.XPD.TOTL.GD.ZS': 'education_expenditure',
    'SE.PRM.ENRL.TC.ZS': 'pupil_teacher_ratio'
}

# Fetch data for all countries from 2018
df_list = []
for code, name in indicators.items():
    data = wb.download(indicator=code, country='all', start=2018, end=2018)
    data = data.rename(columns={code: name})
    df_list.append(data)

# Merge all dataframes on index (country, year)
final_df = pd.concat(df_list, axis=1).reset_index()

# Drop missing values
final_df.dropna(inplace=True)

# Preview
print(final_df.head())


C:\Users\user\AppData\Local\Temp\ipykernel_33408\3084217856.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = wb.download(indicator=code, country='all', start=2018, end=2018)
C:\Users\user\AppData\Local\Temp\ipykernel_33408\3084217856.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = wb.download(indicator=code, country='all', start=2018, end=2018)
C:\Users\user\AppData\Local\Temp\ipykernel_33408\3084217856.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = wb.download(indicator=code, country='all', start=2018, end=2018)
C:\Users\user\AppData\Local\Temp\ipykernel_33408\3084217856.py:17: FutureWarning: errors='igno

                                       country  year  gdp_per_capita  \
4               Central Europe and the Baltics  2018    16181.607370   
5                   Early-demographic dividend  2018     3503.213201   
6                          East Asia & Pacific  2018    11285.897230   
7  East Asia & Pacific (excluding high income)  2018     7925.424765   
8   East Asia & Pacific (IDA & IBRD countries)  2018     8014.497663   

   literacy_rate  primary_enrollment  secondary_enrollment  \
4      99.418678           95.326233            101.131561   
5      79.297913          101.392952             74.488922   
6      95.369431          100.902206             87.774780   
7      95.632133          100.901573             86.334831   
8      95.577133          100.975578             86.210693   

   education_expenditure  pupil_teacher_ratio  
4               4.243570             12.66168  
5               3.689163             27.85282  
6               3.261967             17.53589  
7 

C:\Users\user\AppData\Local\Temp\ipykernel_33408\3084217856.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = wb.download(indicator=code, country='all', start=2018, end=2018)


In [11]:
wb.download(indicator='NY.GDP.PCAP.CD', country='all', start=2018, end=2018)

C:\Users\user\AppData\Local\Temp\ipykernel_33408\2507428105.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  wb.download(indicator='NY.GDP.PCAP.CD', country='all', start=2018, end=2018)


,,NY.GDP.PCAP.CD
country,year,
Africa Eastern and Southern,2018,1538.924188
Africa Western and Central,2018,1703.896392
Arab World,2018,6576.516631
Caribbean small states,2018,12047.852811
Central Europe and the Baltics,2018,16181.607370
...,...,...
Virgin Islands (U.S.),2018,36663.208755
West Bank and Gaza,2018,3562.330943
"Yemen, Rep.",2018,633.887206
